Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [0]:
import pandas as pd

## Import .csv from API calls
STATIONS= pd.read_csv('../data/city_bikes.csv')
YELP = pd.read_csv('../data/Yelp_location_results.csv')
FS = pd.read_csv('../data/FS_location_results.csv')

In [ ]:
### EXPLORATION AND TESTING OF PRIMARY KEY BETWEEN TABLES. 

#Architecture 
# - STATIONS will be the LEFT TABLE as it is the 'master' list of station id and the lookup key.
# - LAT/LONG for the STATION will be the primary key
# - JOIN dataframes on LAT/LONG. Anticipate there will be multiple results per station id (1-many)


# STATIONS.drop_duplicates(subset='lat_long',keep="first").count()

## Add primary key to Yelp based on the Lat/Long of the associated station coords
## Convert it to string or it will just ADD them! doh

YELP['lat_long']=YELP['latitude'].astype(str) +","+ YELP['longitude'].astype(str)
YELP['source']="yelp"
YELP_df = YELP[['name','station_id','rating','price','lat_long','distance','source']]
YELP_df
## Confirm that generated pkey ('lat_long' can be used to join tables. 
## Postive outcome: use outpt of concatenated field as input to filter the Station list. 
## Result : success. 
## Conclusion: Use the generated 'lat_long' as a primary key to join tables. 

# Sample Lat_long from Yelp='49.262487,-123.114397'
# 
# STATIONS_filter = STATIONS[STATIONS['lat_long']=='49.262487,-123.114397']
# STATIONS_filter




In [101]:
### CREATE FOREIGN KEYS IN YELP AND FS DATAFRAMES

## Eliminate extra fields in STATIONS
STATIONS_abr = STATIONS[['lat_long','name','free_bikes']]
STATIONS_abr


YELP['lat_long']=YELP['latitude'].astype(str) +","+ YELP['longitude'].astype(str)
YELP['source']="yelp"
YELP_df = YELP[['name','station_id','rating','price','lat_long','distance','source','id']]
YELP_df
## QA - CHECK - 12,120 records. Lat_long Generated. 

FS['lat_long']=FS['latitude'].astype(str) +","+ FS['longitude'].astype(str)
FS['source']="FS"
FS_df = FS[['name','station_id','rating','price','lat_long','distance','source','fsq_id']]
FS_df

## QA - Observation - FS may have limited the results as there is an curious consistency about the max number of results per STATION
## todo: investigate why there might be a max of then bars per stationid in the FS results 
##FS.groupby('station_id')['name'].count().sort_values(ascending=False)


,name,station_id,rating,price,lat_long,distance,source,fsq_id
0,Rogue Kitchen & Wetbar,7a19c49f486d7c0c02b3685d7b240448,8.2,3.0,"49.262487,-123.114397",250,FS,4f60fc8be4b0a9d090a1f850
1,33 Acres Brewing Co,7a19c49f486d7c0c02b3685d7b240448,9.4,2.0,"49.262487,-123.114397",674,FS,51b201d27dd249ae714ba728
2,The Juice Truck,7a19c49f486d7c0c02b3685d7b240448,8.8,1.0,"49.262487,-123.114397",779,FS,537e3ec8498ec730b9966b97
3,Biercraft Bistro,7a19c49f486d7c0c02b3685d7b240448,8.1,3.0,"49.262487,-123.114397",744,FS,4cbf89f200d8370403c8445c
4,Faculty Brewing Co,7a19c49f486d7c0c02b3685d7b240448,8.3,2.0,"49.262487,-123.114397",994,FS,55d97956498e2c299b5830c0
...,...,...,...,...,...,...,...,...
2259,The Fringe Cafe,cc25ae4f093b33ba0afd1dbc0dd20324,5.8,2.0,"49.265442,-123.187738",955,FS,4aa7f754f964a520484e20e3
2260,Coppertank Grill,cc25ae4f093b33ba0afd1dbc0dd20324,5.7,1.0,"49.265442,-123.187738",941,FS,4aa6a91df964a520864a20e3
2261,Good Co. Kits,cc25ae4f093b33ba0afd1dbc0dd20324,NaN,1.0,"49.265442,-123.187738",791,FS,6457401321f70230178ea1ea
2262,Castaway Bar and Kitchen,cc25ae4f093b33ba0afd1dbc0dd20324,NaN,2.0,"49.265442,-123.187738",809,FS,5d29827cf5c8330023780aff


# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [132]:
import sqlite3
from sqlite3 import Error


def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("connection to sq3 db successful")
    except Error as e:
        print(f"The error '{e}' happened")

    return connection

## Create a CONNECTION between python and the sqlite file 'sm_app'
connection = create_connection('../data/vancouver_bikes')

connection to sq3 db successful


In [ ]:

## Create TABLES
'''
users
posts
comments
likes

TO RUN A QUERY (to create tables) you need a function. 
'''

def execute_qry(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"Computer reported '{e}' error")


#### QUERIES ######

create_user_tble = """
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER,
    gender TEXT,
    nationality TEXT
    ); 
"""

Look at the data before and after the join to validate your data.